# Packages Installation

In [1]:
# Install External packages not available on Colab
!pip install git+https://github.com/HakaiInstitute/hakai-api-client-python.git
!pip install git+https://github.com/HakaiInstitute/ioos_qc@colab-compatible
!pip install git+https://github.com/HakaiInstitute/hakai-profile-qaqc.git
!pip install ipywidgets

  Cloning https://github.com/HakaiInstitute/hakai-api-client-python.git to /tmp/pip-req-build-vc8ogxce
  Running command git clone -q https://github.com/HakaiInstitute/hakai-api-client-python.git /tmp/pip-req-build-vc8ogxce
  Created wheel for hakai-api: filename=hakai_api-0.2.0-cp37-none-any.whl size=2813 sha256=835037f78f8c7fee74d749b0fb549f315897e7605befc18ca4bab9272457469e
  Stored in directory: /tmp/pip-ephem-wheel-cache-1yas6d4g/wheels/c3/a6/1a/0543f35a365c82e28bbc73e6e41ad76cf0d548877ab582cb6f
Successfully built hakai-api
  Cloning https://github.com/HakaiInstitute/ioos_qc (to revision colab-compatible) to /tmp/pip-req-build-2y_6jk8r
  Running command git clone -q https://github.com/HakaiInstitute/ioos_qc /tmp/pip-req-build-2y_6jk8r
  Running command git checkout -b colab-compatible --track origin/colab-compatible
  Switched to a new branch 'colab-compatible'
  Branch 'colab-compatible' set up to track remote branch 'colab-compatible' from 'origin'.
     |███████████████████████

In [1]:
import pandas as pd
import xarray as xr
import datetime as dt

from hakai_api import Client
from ioos_qc.config import QcConfig
import hakai_profile_qc
import ipywidgets as widgets
from ipywidgets import interact

!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


# Retrieve QC Log
This section download the QC Log from the Hakai Database, and compare the manually QCed data versus the QARTOD Flags.

In [ ]:
df_station,url,metadata = hakai_profile_qc.get.hakai_ctd_data('limit=-1',endpoint=qc_endpoint)

In [126]:
# Get QC Log
qc_endpoint = 'eims/views/output/ctd_qc'
df_qc, query_url, unused = hakai_qc.get.hakai_ctd_data('limit=-1',
                                                       endpoint=qc_endpoint)
# Set Index by site and hakai id
df_qc = df_qc.set_index(['site_id','hakai_id']).sort_index()

In [18]:
# Filter data which are associated with a review flag AV, SVC, or SVD
df_qc = df_qc[df_qc.isin(['AV','SVC','SVD']).any(axis='columns')]
df_qc

action  ... rinko_do_ml_l_flag
site_id hakai_id                                ...                   
BU1     018032_2018-07-25T19:17:32.667Z         ...               None
        018032_2019-01-24T16:44:26.834Z         ...               None
        018066_2018-01-25T22:55:34.000Z         ...               None
        01907467_2018-05-01T20:19:26Z           ...               None
        01907467_2018-06-07T19:29:20Z           ...               None
...                                        ...  ...                ...
QU39    080217_2019-09-04T18:37:03.000Z         ...                 AV
        080217_2019-09-24T18:49:12.833Z         ...                 AV
        080217_2019-10-12T20:06:11.500Z         ...                 AV
        080217_2019-10-18T17:29:25.167Z         ...                 AV
        080217_2019-10-30T17:21:52.167Z         ...                SVC

[3942 rows x 31 columns]

In [106]:
# Loop through each of the sites and run qc_config on them and merge to manually qc dataframe
station='BU1'
qartod_full_result = pd.DataFrame()

for station,df_station in df_qc.groupby(level=0):
    print(station)
    hakai_id = df_station.loc[station].index.tolist()
    df,url, metadata = hakai_qc.get.hakai_ctd_data('station=' + station +
                                                '&hakai_id={'+ ','.join(hakai_id)+'}'+
                                                '&limit=-1')

    # Sort values and get derived variables
    group_variables = ['device_model','device_sn','ctd_file_pk','ctd_cast_pk','direction_flag']
    df = df.sort_values(by=group_variables+['pressure'])
    df = hakai_qc.utils.derived_ocean_variables(df)

    # Load qc_config and run QC
    qc_config = hakai_qc.get.json_config('hakai_ctd_profile.json')
    hakai_stations = hakai_qc.get.hakai_stations()
    df_qartod = hakai_qc.run.tests_on_profiles(df,hakai_stations,qc_config)
    df_qartod = df_qartod[df_qartod['direction_flag']=='d']

    # Get flag statistics for each hakai_id
    qartod_result = pd.DataFrame()
    for var in set(df_qartod.filter(like='_qartod_flag').columns):
        df_temp = (df_qartod.groupby(['station','hakai_id',var]).count()['depth']\
            /df_qartod.groupby(['station','hakai_id']).count()['depth'])\
            .unstack(level=-1)
        df_temp.columns = pd.MultiIndex.from_product([[var],df_temp.columns.to_list()])
        if len(qartod_result)!=0:
            qartod_result = qartod_result.merge(
                df_temp,how='outer',
                right_index=True,left_index=True)
        else:
            qartod_result = df_temp

    qartod_full_result = qartod_full_result.append(qartod_result)

BU1
QAQC BU1
  position
    qartod
      ('location_test', {'bbox': [-180, -90, 180, 90], 'target_range': 3000, 'target_lat': [50.370000000000104], 'target_lon': [-125.08]})
  pressure
    qartod
      ('gross_range_test', {'suspect_span': [0, 641.7098932401286], 'fail_span': [0, 672.2675072039443], 'maximum_suspect_depth_ratio': 1.05, 'maximum_fail_depth_ratio': 1.1})
  depth
    qartod
      ('gross_range_test', {'suspect_span': [0, 635.25], 'fail_span': [0, 665.5], 'maximum_suspect_depth_ratio': 1.05, 'maximum_fail_depth_ratio': 1.1})
  dissolved_oxygen_ml_l
    qartod
      ('gross_range_test', {'fail_span': [0, 20], 'suspect_span': [1, 15]})
      ('rate_of_change_test', {'threshold': 3})
      ('spike_test', {'suspect_threshold': 0.5, 'fail_threshold': 1})
      ('attenuated_signal_test', {'suspect_threshold': 0.1, 'fail_threshold': 0.01, 'check_type': 'range'})
  dissolved_oxygen_percent
    qartod
      ('gross_range_test', {'fail_span': [-1, 150], 'suspect_span': [0, 140]})
  

KeyboardInterrupt: ignored

In [99]:
qartod_result = pd.DataFrame()
for var in set(df_qartod.filter(like='_qartod_flag').columns):
    df_temp = (df_qartod.groupby(['station','hakai_id',var]).count()['depth']\
        /df_qartod.groupby(['station','hakai_id']).count()['depth'])\
        .unstack(level=-1)
    df_temp.columns = pd.MultiIndex.from_product([[var],df_temp.columns.to_list()])
    if len(qartod_result)!=0:
        qartod_result = qartod_result.merge(
            df_temp,how='outer',
            right_index=True,left_index=True)
    else:
        qartod_result = df_temp


In [122]:
qartod_full_result.index.names = ['site_id','hakai_id']
compare_qc = df_qc.merge(qartod_full_result,left_index=True,right_index=True,how='outer')

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning:

merging between different levels can give an unintended result (1 levels on the left,2 on the right)



In [123]:
compare_qc

action  ... (turbidity_qartod_flag, 4.0)
site_id hakai_id                                ...                             
BU1     018032_2018-07-25T19:17:32.667Z         ...                          NaN
        018032_2019-01-24T16:44:26.834Z         ...                          NaN
        018066_2018-01-25T22:55:34.000Z         ...                          NaN
        01907467_2018-05-01T20:19:26Z           ...                          NaN
        01907467_2018-06-07T19:29:20Z           ...                          NaN
...                                        ...  ...                          ...
QU39    080217_2019-09-04T18:37:03.000Z         ...                          NaN
        080217_2019-09-24T18:49:12.833Z         ...                          NaN
        080217_2019-10-12T20:06:11.500Z         ...                          NaN
        080217_2019-10-18T17:29:25.167Z         ...                          NaN
        080217_2019-10-30T17:21:52.167Z         ...                          NaN

[3942 rows x 73 columns]

In [ ]:
# Get Hakai CTD Data Download through the API
station = 'QU39'
start_time = '2019-01-01'
end_time = '2020-01-01'

variable_lists = hakai_qc.get.hakai_api_selected_variables()

# Let's just get the data from QU39
filterUrl = 'station='+station+'&status!=MISCAST&limit=-1'+\
    '&measurement_dt>'+start_time+'&measurement_dt<'+end_time+\
    '&fields='+','.join(variable_lists)
df, url = hakai_qc.get.hakai_ctd_data(filterUrl)
print(str(len(df))+' records found')

# Regroup profiles and cast direction and sort them by pressure
group_variables = ['device_model','device_sn','ctd_file_pk','ctd_cast_pk','direction_flag']
df = df.sort_values(by=group_variables+['pressure'])

# Just show the first few lines to have a look
df.head() # Show the top of the data frame

Please go here and authorize:
https://hecate.hakai.org/api/auth/oauth2?response_type=code&client_id=289782143400-1f4r7l823cqg8fthd31ch4ug0thpejme.apps.googleusercontent.com&state=3dn5sYBwITH3CU7JQljgUWULV4kIJU

Paste the full redirect URL here:
https://hecate.hakai.org/api/auth/oauth2/callback?state=3dn5sYBwITH3CU7JQljgUWULV4kIJU&code=4/0AY0e-g49mK552g5Z8ZVETyNaK6LLb4O1tEl1h3IafK6og3Zyv7wIf-iCOYbt-ui2Sj_rZw&scope=email%20profile%20openid%20https://www.googleapis.com/auth/userinfo.email%20https://www.googleapis.com/auth/userinfo.profile&authuser=1&hd=hakai.org&prompt=none
16162 records found


,ctd_file_pk,ctd_cast_pk,hakai_id,ctd_data_pk,filename,device_model,device_sn,work_area,cruise,station,cast_number,latitude,longitude,start_dt,bottom_dt,end_dt,duration,start_depth,bottom_depth,direction_flag,measurement_dt,descent_rate,conductivity,conductivity_flag,temperature,temperature_flag,depth,depth_flag,pressure,pressure_flag,par,par_flag,flc,flc_flag,turbidity,turbidity_flag,ph,ph_flag,salinity,salinity_flag,spec_cond,spec_cond_flag,dissolved_oxygen_ml_l,dissolved_oxygen_ml_l_flag,rinko_do_ml_l,rinko_do_ml_l_flag,dissolved_oxygen_percent,dissolved_oxygen_percent_flag,oxygen_voltage,oxygen_voltage_flag,c_star_at,c_star_at_flag,sos_un,sos_un_flag,backscatter_beta,backscatter_beta_flag,cdom_ppb,cdom_ppb_flag
5237,5505,13723,080217_2019-04-02T19:41:11.666Z,8614263,080217_20190402_1500,RBRmaestro,080217,QUADRA,QOMA5,QU39,3,50.0299,-125.100205,2019-04-02T19:41:12.000Z,2019-04-02T19:44:50.000Z,2019-04-02T19:49:06.166Z,218,1.223077,220.568071,d,2019-04-02T19:41:12.833Z,0.461,32.18067,None,10.0965,None,2.036,None,2,None,189.784,None,0.5595,None,1.254,None,None,None,28.9465,None,None,None,9.9649,None,NaN,None,151.9723,None,NaN,None,NaN,None,None,None,None,None,None,None
5238,5505,13723,080217_2019-04-02T19:41:11.666Z,8614264,080217_20190402_1500,RBRmaestro,080217,QUADRA,QOMA5,QU39,3,50.0299,-125.100205,2019-04-02T19:41:12.000Z,2019-04-02T19:44:50.000Z,2019-04-02T19:49:06.166Z,218,1.223077,220.568071,d,2019-04-02T19:41:14.250Z,0.915,32.09487,None,9.9572,None,2.970,None,3,None,163.194,None,0.6324,None,1.230,None,None,None,28.9720,None,None,None,10.0968,None,NaN,None,154.1065,None,NaN,None,NaN,None,None,None,None,None,None,None
5239,5505,13723,080217_2019-04-02T19:41:11.666Z,8614265,080217_20190402_1500,RBRmaestro,080217,QUADRA,QOMA5,QU39,3,50.0299,-125.100205,2019-04-02T19:41:12.000Z,2019-04-02T19:44:50.000Z,2019-04-02T19:49:06.166Z,218,1.223077,220.568071,d,2019-04-02T19:41:15.333Z,0.819,31.94162,None,9.6349,None,3.890,None,4,None,148.817,None,0.6346,None,1.138,None,None,None,29.0775,None,None,None,10.1723,None,NaN,None,154.0950,None,NaN,None,NaN,None,None,None,None,None,None,None
5240,5505,13723,080217_2019-04-02T19:41:11.666Z,8614266,080217_20190402_1500,RBRmaestro,080217,QUADRA,QOMA5,QU39,3,50.0299,-125.100205,2019-04-02T19:41:12.000Z,2019-04-02T19:44:50.000Z,2019-04-02T19:49:06.166Z,218,1.223077,220.568071,d,2019-04-02T19:41:16.416Z,1.037,31.68160,None,9.1662,None,4.906,None,5,None,153.532,None,0.5052,None,1.184,None,None,None,29.1948,None,None,None,10.1846,None,NaN,None,153.2516,None,NaN,None,NaN,None,None,None,None,None,None,None
5241,5505,13723,080217_2019-04-02T19:41:11.666Z,8614267,080217_20190402_1500,RBRmaestro,080217,QUADRA,QOMA5,QU39,3,50.0299,-125.100205,2019-04-02T19:41:12.000Z,2019-04-02T19:44:50.000Z,2019-04-02T19:49:06.166Z,218,1.223077,220.568071,d,2019-04-02T19:41:17.500Z,0.940,31.54936,None,8.9673,None,5.986,None,6,None,173.030,None,0.4477,None,1.140,None,None,None,29.2222,None,None,None,10.1087,None,NaN,None,151.2991,None,NaN,None,NaN,None,None,None,None,None,None,None


# Review Profile
Let's have a look at the percentage of flags present for each tests.

In [ ]:
# Get the list of parameters (variables associated with a *_flag column), ignore direction_flag
flag_columns = df.filter(like='_flag').columns.tolist()
var_columns = set([sub.replace('_flag','') for sub in flag_columns])-{'direction'}

# Temporary until the database has the right setup:
for var in var_columns:
    if var+'_qartod_flag' not in df.columns:
        df[var+'_qartod_flag'] = 2
    if var+'_flag_description' not in df.columns:
        df[var+'_flag_description'] = ''
        
# Retrive list of tests
test_list = sorted(df.filter(regex='_flag$|_test$').columns.tolist())
test_list.append("None")

# Run the interactive profile review tool
hakai_qc.show.interactive_profile_viewer(df,var_columns,test_list)

Select Flagged Variable(s) to consider in the following list:


interactive(children=(Dropdown(description='Test to review', index=37, options=('backscatter_beta_flag', 'back…

# Define Standard Attributes


In [ ]:
# Your coordinates
creator = {
    'creator_name': 'Jessy Barrette',
    'creator_url': 'https://hakai.org/',
    'creator_email': 'jessy.barrette@hakai.org',
    'creator_institution':'Hakai Institute',
    'creator_country':'Canada'
}
# Hakai Coordinates
publisher = {
    'publisher_name': 'Hakai Institute',
    'publisher_url': 'https://hakai.org/',
    'publisher_email': 'contact@hakai.org',
    'publisher_country':'Canada',
    'publisher_institution':'Hakai Institute'
}
# General Global Attributes related to this dataset
standard_global_attributes = {'institution':'Hakai Institute',
                    'project':'Hakai Oceanography',
                    'summary':'text describing that specific data',
                    'comment': '',
                    'infoUrl':'https://hakai.org/',
                    'keywords':'conductivity,temperature,salinity,depth,pressure,dissolved oxygen',
                    'acknowledgment':'Hakai Field Techniciens, research and IT groups',
                    'naming_authority':'Hakai Instititute',
                    'standard_name_vocabulary': 'CF Standard Name Table v1.3',
                    'license':'unknown',
                    'geospatial_lat_units':'degrees_north',
                    'geospatial_lon_units':'degrees_east'}
# Standard Variable Attributes
standard_variable_attributes = {}


# Generate a NetCDF File for the approuved profiles!

In [ ]:
approuved_hakai_id = ['080217_2017-01-08T18:03:05.167Z','080217_2017-01-26T16:56:39.000Z']

save_to = ''
for hakai_id in approuved_hakai_id:
    hakai_id_global_attributes = {'title':'Hakai Research CTD Profile: '+hakai_id,
                         'id':hakai_id,
                         'date_created':str(dt.datetime.utcnow().isoformat()),
                         }
    global_attributes = standard_global_attributes 
    global_attributes.update(hakai_id_global_attributes)
    global_attributes.update(creator)
    global_attributes.update(publisher)

    # Create the NetCDF Source file!
    hakai_qc.get.research_profile_netcdf(hakai_id,save_to,
                                         extra_global_attributes=global_attributes,
                                         extra_variable_attributes=standard_variable_attributes)

# To Download Generated Files 
Click on the folder icon on the left bar of google colab and select the files to download.
